## **앱스토어 리뷰 감성 분석**

### 문제정의
각 앱들의 리뷰들과 평점이 양의 관계에 있다고 가정하고 학습을 진행하여 평점을 예측

### 데이터 불러오기

tsv 파일 불러오기

In [2]:
def get_data_split(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f.readlines()]
        docs = [doc for doc in docs if len(doc) == 3]
        
    if not docs:
        app_ids, texts, ratings = [], [], []

    app_ids, texts, ratings = zip(*docs)
    return list(app_ids), list(texts), list(ratings)

In [3]:
train_app_ids, train_texts, train_ratings = get_data_split('./data/train_docs.txt')
test_app_ids, test_texts, test_ratings = get_data_split('./data/test_docs.txt')

json 파일 불러오기

In [1]:
import ujson

def get_data_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        texts = file.read()
        data = ujson.loads(texts)
        app_ids_list = list(data.keys())
    return data, app_ids_list

In [55]:
train_data, train_app_id_list = get_data_json('./data/train_json.txt')
test_data, test_app_id_list = get_data_json('./data/test_json.txt')

### 데이터 전처리: 형태소 분석
1. 축약어, 오타:
골치거리임, 전형적인 한국 사전에 없는 단어들도 많음, 사전에 없는 단어를 트위터사전에 넣어야 할 듯
2. 띄어쓰기가 불분명함

$\Rightarrow$ data Cleaning class 필요성 있음

### 데이터 후처리

데이터 후처리란 형태소 분석기를 거친 후에 이상한 단어들, 신조어 등을 처리하는 작업


각 앱별로 공백문자를 나눠서 공통된 단어를 추출해보기? 앱별로 공통으로 많이 이야기 하는 단어들이 제대로 형태소 분석이 되는지 테스트 하기 위해 단어들을 추출해본다

문제점: 일일이 확인 하기 어려움

In [2]:
from collections import Counter, defaultdict
from konlpy.tag import Twitter, Komoran, Mecab
twitter = Twitter()
komoran = Komoran()
mecab = Mecab()

In [75]:
def get_words_count(data, app_id_list, tokenizer, app_idx='random'):
    if app_idx == 'random':
        idx = np.random.choice(len(app_id_list))
        print(idx)
        texts = get_reviews_by_id(data, app_id_list, idx=idx)
    elif type(app_idx) == int: # write a num
        texts = get_reviews_by_id(data, app_id_list, idx=app_idx)
    
    word_count = Counter()
    for review in texts:
        if tokenizer.__class__ == Twitter:
            tokens = tokenizer.pos(review, norm=True, stem=True)
            tokens_list = ['/'.join(token) for token in tokens]
            word_count.update(tokens_list)
        else:
            tokens = tokenizer.pos(review)
            tokens_list = ['/'.join(token) for token in tokens]
            word_count.update(tokens_list)
    
    return word_count

def get_reviews_by_id(data, app_id_list, idx):
    """app_id별로 텍스트를 가져온다"""
    app_id = app_id_list[idx]
    texts = data[app_id]['reviews']
    return texts

In [94]:
word_count = get_words_count(train_data, train_app_id_list, twitter, 0)

## 형태소 분석 후처리

### Mecab [잠정보류]

In [4]:
train_ma_docs, train_app_id_list = get_data_json('./data/ma_train_json_mecab.txt')

In [5]:
train_ma_docs[train_app_id_list[0]][2]

[['해외', 'NNG'],
 ['에서', 'JKB'],
 ['도', 'JX'],
 ['쓸', 'VV+ETM'],
 ['수', 'NNB'],
 ['있', 'VV'],
 ['게', 'EC'],
 ['해', 'VV+EC'],
 ['주', 'VX'],
 ['세요', 'EP+EF'],
 ['ㅠㅠㅠㅠ', 'UNKNOWN']]

### Unknown에 대한 처리(Mecab) [잠정보류]
부호같은 거는 후처리를 안하고 그냥 제거하기로 

In [6]:
def get_unknown_words(ma_docs, app_id_list):
    unknown_list = []
    for app_id in app_id_list:
        for doc in ma_docs[app_id]:
            for ma in doc:
                if ma[1] in ['UNKNOWN', 'NA']:
                    unknown_list.append(ma[0])
    return list(set(unknown_list))

In [7]:
unknown_list = get_unknown_words(train_ma_docs, train_app_id_list)
len(unknown_list)

17237

In [10]:
unknown_list[40:60]

['ㅓ테',
 '퀄에',
 'ㅑ어ㅓ어어울ㄹ',
 'ㅇㄹㅊ',
 'ㅠㅠ여행가서',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이렇게',
 'ㅓㅏㅇㅈ랗',
 '꿧어요',
 'ㅠㅠㅠ영구정지좀제발풀어주세요ㅠㅠ',
 '둿는대',
 'ㅡㅡ어이없네',
 'ㅡㅡ얼른',
 '뎃된지가언젠데',
 'ㅛㅗ',
 '꿧네요',
 '뷴기다려야함',
 'ㅋㅋㅋㅋㅋ편하수',
 '됫는지',
 'ㅋㅋ나잇값하자',
 'ㅠㅠ영어사전']

주로 말도 안되는 말들이나 ㅡ, ㅉ,ㅜ,ㅠ,ㅋ 앞이나 뒤에 붙어서 형태소 분석이 안된다.( 웃음(ㅋㅋ)이나 이모티콘(ㅜㅜ) 같은..) 이 단어들을 때주고 다시 넣어보자

처리가 되면 

{'기존 unknown':'형태소 분석 된 unknown'} 과 잔존 unkown_list 출력

앞에의 dictionary를 ma_unknown_rule.txt로 저장해서 두고주고 쓸것

In [12]:
import unicodedata

In [44]:
def remove_emo(unknown_list, tokenizer):
    """UNKOWN 단어들의 앞뒤에 ㅡ,ㅉ,ㅠ,ㅜ,ㅋ 등등 를 제거하고 형태소 분석 결과 내보냄"""
    rest_unknown_list = []
    unknown_rule = defaultdict()
    for unknown_word in unknown_list:
        new_word, rest = check_word(unknown_word)
        tokens = tokenizer.pos(new_word)
        if len(tokens) > 1:
            unknown_rule[unknown_word] = tokens
        elif (len(tokens) == 1) and tokens[0][1] != 'UNKNOWN':
            unknown_rule[unknown_word] = tokens
        else:
            rest_unknown_list.append(unknown_word)
        
    return unknown_rule, rest_unknown_list


def check_word(word):
    new_word = []
    rest = []
    for char in word:
        if check_jamo(char):
            rest.append(char)
        else:
            new_word.append(char)
    new_word = ''.join(new_word)
    
    return new_word, rest

def check_jamo(char):
    CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    if (char in CHOSUNG_LIST) | (char in JUNGSUNG_LIST) | (char in JONGSUNG_LIST):
        return True
    else:
        return False

In [37]:
unknown_rule, rest_unknown_list = remove_emo(unknown_list, mecab)

In [38]:
len(rest_unknown_list)

9981

---

## Twitter

트위터가 더 잘된거 같기도.. '내역'만 좀 어떻게 해보자

In [3]:
train_ma_docs, app_id_list = get_data_json('./data/ma_train_json_twitter_norm.txt')

In [4]:
import itertools

def find_sublists(seq, sublist):
    length = len(sublist)
    for index, value in enumerate(seq):
        if value == sublist[0] and seq[index:index+length] == sublist:
            yield index, index+length
            
def replace_sublist(seq, target, replacement, maxreplace=None):
    sublists = find_sublists(seq, target)
    if maxreplace:
        sublists = itertools.islice(sublists, maxreplace)
    for start, end in sublists:
        seq[start:end] = replacement
        
def replace_ma_docs(ma_docs, app_id_list, post_ma_pairs):
    for app_id in app_id_list:
        docs = ma_docs[app_id]
        for doc in docs:
            for ma_set in post_ma_pairs:
                replace_sublist(doc, ma_set[0], ma_set[1])
    return ma_docs

In [5]:
# [([before], [after]), ([before], [after]), () ...]
ma_pairs = [
            ([['내', 'Determiner'], ['역', 'Noun']], [['내역', 'Noun']]),
            ([['징', 'Noun'], ['그럽', 'Adjective']], [['징그럽', 'Adjective']])
           ]

In [6]:
train_ma_docs = replace_ma_docs(train_ma_docs, app_id_list, ma_pairs)

save change rule

In [7]:
from utils import Post_ma

In [8]:
post_ma_pairs = Post_ma()

In [9]:
post_ma_pairs.update(ma_pairs)

There are duplicated rules! find you input pairs in index of [0, 1]
update num: 2 


### 주요 품사별 추출

MNG(일반 명사), NNP(고유 명사), VV(동사), VA(형용사)이며, 경우에 따라 MM(관형사), MAG(일반 부사), MAJ(접속 부사)

무엇을 추출할지 결정해보자, 제거할 것. Eomi, PreEomi, Punctuation

[[['맨날', 'Adverb'],
  ['보여', 'Verb'],
  ['준', 'Verb'],
  ['것', 'PreEomi'],
  ['만', 'Eomi'],
  ['계속', 'Noun'],
  ['돌아가면', 'Verb'],
  ['서', 'Eomi'],
  ['보여', 'Verb'],
  ['주', 'PreEomi'],
  ['는군', 'Eomi'],
  ['..', 'Punctuation'],
  ['징그럽', 'Adjective'],
  ['다', 'Eomi'],
  ['진짜', 'Noun']],
 [['잘', 'Verb'],
  ['보고', 'Noun'],
  ['있었', 'Adjective'],
  ['는데', 'Eomi'],
  ['갑자기', 'Noun'],
  ['없어진', 'Verb'],
  ['이유', 'Noun'],
  ['가', 'Josa'],
  ['뭐', 'Noun'],
  ['죠', 'Josa'],
  ['??????????????!!!?', 'Punctuation']],
 [['해외', 'Noun'],
  ['에서도', 'Josa'],
  ['쓸', 'Verb'],
  ['수', 'PreEomi'],
  ['있', 'PreEomi'],
  ['게', 'Eomi'],
  ['해주세', 'Verb'],
  ['요', 'Eomi'],
  ['ㅠㅠ', 'KoreanParticle']],
 [['세기', 'Noun'],
  ['의', 'Josa'],
  ['경기', 'Noun'],
  ['는', 'Josa'],
  ['나오지', 'Verb'],
  ['않았', 'Verb'],
  ['다고', 'Eomi'],
  ['한', 'Verb'],
  ['다', 'Eomi'],
  ['.', 'Punctuation']],
 [['물론', 'Noun'],
  ['특별한', 'Adjective'],
  ['이벤트', 'Noun'],
  ['가', 'Josa'],
  ['있어', 'Adjective'],
  ['서', 'Eomi'],
  ['이용자',